[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Kaggle-runa/MameLand_vol3/blob/main/src/notebook/05_%E9%A6%AC%E5%88%B8%E3%81%AE%E8%B3%BC%E5%85%A5%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3(%E3%83%AF%E3%82%A4%E3%83%89%E3%83%BB%E8%A4%87%E5%8B%9D).ipynb
)

In [ ]:
# 必要なライブラリのimport
import numpy as np
import pandas as pd

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

In [ ]:
# google driveへのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


データはGoogle Driveの競馬分析/dataレポジトリにあることを想定しています。  
自分のフォルダ構成に応じてデータのパスを適宜変更して下さい。


- 競馬分析/
  - data/  # 分析に使う生データ
  - feature_data/  # 02_データの前処理.ipynbで作成した生データを加工したデータ
  - simulation_data/ # 05_馬券の購入シミュレーション(ワイド・複勝).ipynbで利用する回収率を計算するためのデータ
  - notebooks/  # 競馬分析を行うnotebook
    - 00_データのスクレイピング.ipynb
    - 01_競馬データ可視化.ipynb
    - 02_データの前処理.ipynb
    - 03_モデルの学習.ipynb
    - 04_新規データでの予測.ipynb
    - 05_馬券の購入シミュレーション(ワイド・複勝).ipynb
  - model/  # 作成したモデルを格納するレポジトリ

In [23]:
# データ読み込み
simulation_data = pd.read_csv('/content/drive/MyDrive/競馬分析/simulation_data/simulation_20241103.csv')
pay_result = pd.read_csv('/content/drive/MyDrive/競馬分析/data/pay_results.csv')

In [48]:
# データの確認
simulation_data.head()

,race_id,horse_number,finish_position,target,pred,pred_class
0,202401020801,1,7.0,0,0.054699,0
1,202401020801,2,5.0,0,0.421522,1
2,202401020801,3,4.0,0,0.334618,0
3,202401020801,4,2.0,1,0.380848,1
4,202401020801,5,8.0,0,0.034661,0


In [49]:
# データの確認
pay_result.head()

,race_id,baken_types,horse_number,refund,popularity
0,201901010101,単勝,1,140,1
1,201901010101,複勝,1br3br4,110br110br470,1br2br7
2,201901010101,枠連,1 - 3,190,1
3,201901010101,馬連,1 - 3,190,1
4,201901010101,ワイド,1 - 3br1 - 4br3 - 4,"120br840br1,100",1br12br13


### 馬券の買い方一覧(WIN5などその他買い方もあります)  
参考サイト：https://www.jra.go.jp/kouza/beginner/baken/

|  名称  |  説明  |
| ---- | ---- |
|  単勝  |  1着になる馬を当てる馬券  |
|  複勝  |  3着までに入る馬を当てる馬券(出走する馬が7頭以下の場合は、2着までが的中)  |
|  馬連  |  1着と2着になる馬の馬番号の組合せを当てる馬券  |
|  馬単  |  1着と2着になる馬の馬番号を着順通りに当てる馬券  |
|  ワイド  |  3着までに入る2頭の組合せを馬番号で当てる馬券 |
|  3連複  |  1着、2着、3着となる馬の組合せを馬番号で当てる馬券  |
|  3連単  |  1着、2着、3着となる馬の馬番号を着順通りに当てる馬券  |

### 参考：競馬の控除率(仮にすべての馬券を買った場合に帰ってくる金額の割合)  
参考サイト：https://db-keiba.com/return-average/#st-toc-h-3  


|  券種  | 控除率 |  払戻率 |
| ---- | ---- | ---- |
| 単勝 |  20.0% | 80.0% |
| 複勝 | 20.0% | 80.0% |
| 馬連 | 22.5% | 77.5% |
| 馬単 | 25.0% | 75.0% |
| ワイド | 25.0% | 75.0% |
| 三連複 | 25.0%  | 75.0% |
| 三連単 | 27.5%  | 72.5% |

→100%を超えるのが理想ですが、まずはそれぞれの馬券の払戻率を超えることを目標としましょう

## 購入方針の検討
1. 03_モデルの学習.ipynbでそれぞれのレースid毎にpredの確率上位３頭にフラグを立てているので、これをもとに馬券を購入する
2. predの閾値を求め、その閾値以上の馬券を購入する

In [50]:
class RacePayKinds:
    def __init__(self, dataframe):
        self.return_tables = dataframe

    def _split_columns(self, dataframe, column_name, sep, new_column_prefix):
        return dataframe[column_name].astype(str).str.split(sep, expand=True).add_prefix(new_column_prefix)

    def _prepare_dataframe(self, dataframe, baken_type, horse_sep, refund_sep=None):
        df = dataframe[dataframe["baken_types"] == baken_type][["horse_number", "refund"]]
        wins = self._split_columns(df, "horse_number", horse_sep, "win_")
        returns = self._split_columns(df, "refund", refund_sep or horse_sep, "return_")
        combined_df = pd.concat([wins, returns], axis=1)
        return combined_df.apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0).astype(int)

    def get_race_results(self, race_id, baken_type, horse_sep, refund_sep=None):
        df = self.return_tables[(self.return_tables["race_id"] == race_id) & (self.return_tables["baken_types"] == baken_type)]
        return self._prepare_dataframe(df, baken_type, horse_sep, refund_sep)

    def get_fukusho(self, race_id):
        df = self.get_race_results(race_id, '複勝', 'br')
        # Transform the dataframe to the desired format
        transformed_df = pd.concat([
            pd.DataFrame({'win': df['win_0'], 'return': df['return_0']}),
            pd.DataFrame({'win': df['win_1'], 'return': df['return_1']})
        ], ignore_index=True)
        if 'win_2' in df.columns and 'return_2' in df.columns:
            transformed_df = pd.concat([transformed_df, pd.DataFrame({'win': df['win_2'], 'return': df['return_2']})], ignore_index=True)
        return transformed_df

    def get_tansho(self, race_id):
        return self.get_race_results(race_id, '単勝', None)

    def get_umaren(self, race_id):
        return self.get_race_results(race_id, '馬連', ' - ')

    def get_umatan(self, race_id):
        return self.get_race_results(race_id, '馬単', '→')

    def get_wide(self, race_id):
        wide_df = self.return_tables[(self.return_tables["race_id"] == race_id) & (self.return_tables["baken_types"] == 'ワイド')]
        wins = wide_df["horse_number"].astype(str).str.split('br', expand=True).stack().str.split(' - ', expand=True).add_prefix('win_')
        returns = wide_df["refund"].astype(str).str.split('br', expand=True).stack().str.replace(',', '').rename('return')
        combined_df = pd.concat([wins.reset_index(drop=True), returns.reset_index(drop=True)], axis=1)
        return combined_df.apply(lambda x: pd.to_numeric(x, errors='coerce')).fillna(0).astype(int)

    def get_sanrentan(self, race_id):
        return self.get_race_results(race_id, '三連単', '→')

    def get_sanrenpuku(self, race_id):
        return self.get_race_results(race_id, '三連複', ' - ')

In [42]:
# データフレームを使ってRacePayKindsクラスをインスタンス化
race_results = RacePayKinds(pay_result)

# 特定のrace_idの複勝の結果を表示
race_results.get_fukusho(202206010101)

,win,return
0,15,210
1,10,1600
2,4,170


In [43]:
# 特定のrace_idのワイドの結果を表示
race_results.get_wide(202206010101)

,win_0,win_1,return
0,10,15,6890
1,4,15,660
2,4,10,5640


In [58]:
# ワイドと複勝の回収率を計算
def recovery_rate(race_results, return_tables, bet="wide"):
    race_pay_kinds = RacePayKinds(return_tables)
    targeted_races = {}
    number_of_hits = 0
    amount_of_recovery = 0
    total_rows = 0

    race_id_list = return_tables["race_id"].unique()

    if bet == "wide":
        for race_id in race_id_list:
            pred_numbers = race_results[(race_results["race_id"] == race_id) & (race_results["pred_class"] == 1)]["horse_number"].to_list()
            data = race_pay_kinds.get_wide(race_id)
            if len(pred_numbers) < 3:
              total_rows += 1
            else:
              total_rows += len(data)

            for index, row in data.iterrows():
                if (row["win_0"] in pred_numbers) and (row["win_1"] in pred_numbers):
                    if race_id not in targeted_races:
                        targeted_races[race_id] = []
                    targeted_races[race_id].append({'return': row["return"], 'horses': [row["win_0"], row["win_1"]]})
                    number_of_hits += 1
                    amount_of_recovery += row["return"]

    if bet == "fukusho":
        for race_id in race_id_list:
            pred_numbers = race_results[(race_results["race_id"] == race_id) & (race_results["pred_class"] == 1)]["horse_number"].to_list()
            data = race_pay_kinds.get_fukusho(race_id)
            if len(pred_numbers) < 3:
              total_rows += len(pred_numbers)
            else:
              total_rows += len(data)

            for index, row in data.iterrows():
                if row["win"] in pred_numbers:
                    if race_id not in targeted_races:
                        targeted_races[race_id] = []
                    targeted_races[race_id].append({'return': row["return"], 'horses': [row["win"]]})
                    number_of_hits += 1
                    amount_of_recovery += row["return"]
    hits_rate = number_of_hits / total_rows
    recovery_rate = amount_of_recovery / (total_rows * 100)

    print('的中したレース')
    for race_id, details in targeted_races.items():
        for detail in details:
            print(f'レースID: {race_id}, 的中馬番: {detail["horses"]}, 回収金: {detail["return"]}')
    print('-'* 100)
    print('予測対象レース数', len(race_id_list))
    print('-'* 100)
    print('予測対象馬券数', total_rows)
    print('-'* 100)
    print('的中数', number_of_hits)
    print('-'* 100)
    print('的中率', str(hits_rate * 100) + '%')
    print('-'* 100)
    print('回収金額', amount_of_recovery)
    print('-'* 100)
    print('回収率', str(recovery_rate * 100) + '%')

### 1. の場合
- モデルは3着以内に入る馬を予測するように設計されています。そのため、これに関連する馬券（複勝・ワイド・3連複）での購入結果をシミュレーションしてみます。

In [59]:
# 予測が1になっているデータのみを取得する
data = simulation_data[simulation_data["pred_class"] == 1]

# 結果データの race_id のリストを取得
result_race_ids = data['race_id'].unique()

# 払戻金データを結果データの race_id のみにフィルタリング
filtered_return_tables = pay_result[pay_result['race_id'].isin(result_race_ids)]

In [60]:
# ワイドの回収率
recovery_rate(data, filtered_return_tables)

的中したレース
レースID: 202401020801, 的中馬番: [4, 8], 回収金: 250
レースID: 202401020807, 的中馬番: [11, 12], 回収金: 290
レースID: 202401020808, 的中馬番: [1, 3], 回収金: 300
レースID: 202401020809, 的中馬番: [4, 5], 回収金: 340
レースID: 202401020811, 的中馬番: [4, 5], 回収金: 370
レースID: 202404030803, 的中馬番: [6, 14], 回収金: 260
レースID: 202404030805, 的中馬番: [3, 5], 回収金: 190
レースID: 202404030806, 的中馬番: [3, 8], 回収金: 180
レースID: 202404030807, 的中馬番: [4, 8], 回収金: 230
レースID: 202404030809, 的中馬番: [1, 7], 回収金: 120
レースID: 202404030809, 的中馬番: [1, 3], 回収金: 170
レースID: 202404030809, 的中馬番: [3, 7], 回収金: 180
レースID: 202404030810, 的中馬番: [9, 11], 回収金: 220
レースID: 202404030811, 的中馬番: [7, 9], 回収金: 240
レースID: 202406040101, 的中馬番: [5, 8], 回収金: 160
レースID: 202406040105, 的中馬番: [3, 4], 回収金: 240
レースID: 202406040107, 的中馬番: [4, 5], 回収金: 250
レースID: 202406040109, 的中馬番: [1, 4], 回収金: 210
レースID: 202406040109, 的中馬番: [1, 2], 回収金: 110
レースID: 202406040109, 的中馬番: [2, 4], 回収金: 190
レースID: 202406040111, 的中馬番: [1, 11], 回収金: 460
レースID: 202406040201, 的中馬番: [2, 4], 回収金: 150
レースID: 202406040201

In [57]:
# 複勝の回収率
recovery_rate(data, filtered_return_tables, bet="fukusho")

的中したレース
レースID: 202401020801, 的中馬番: [8], 回収金: 110
レースID: 202401020801, 的中馬番: [4], 回収金: 150
レースID: 202401020802, 的中馬番: [9], 回収金: 210
レースID: 202401020803, 的中馬番: [10], 回収金: 150
レースID: 202401020806, 的中馬番: [1], 回収金: 230
レースID: 202401020807, 的中馬番: [11], 回収金: 140
レースID: 202401020807, 的中馬番: [12], 回収金: 160
レースID: 202401020808, 的中馬番: [1], 回収金: 120
レースID: 202401020808, 的中馬番: [3], 回収金: 160
レースID: 202401020809, 的中馬番: [4], 回収金: 120
レースID: 202401020809, 的中馬番: [5], 回収金: 170
レースID: 202401020810, 的中馬番: [14], 回収金: 160
レースID: 202401020811, 的中馬番: [4], 回収金: 170
レースID: 202401020811, 的中馬番: [5], 回収金: 150
レースID: 202401020812, 的中馬番: [5], 回収金: 150
レースID: 202404030801, 的中馬番: [4], 回収金: 110
レースID: 202404030802, 的中馬番: [8], 回収金: 130
レースID: 202404030803, 的中馬番: [14], 回収金: 110
レースID: 202404030803, 的中馬番: [6], 回収金: 170
レースID: 202404030804, 的中馬番: [18], 回収金: 250
レースID: 202404030805, 的中馬番: [5], 回収金: 150
レースID: 202404030805, 的中馬番: [3], 回収金: 110
レースID: 202404030806, 的中馬番: [8], 回収金: 110
レースID: 202404030806, 的中馬番: [3], 回収金: 140
レー

### 2. の場合
閾値を0.7以上、0.8以上などで設けて予測対象馬が１頭の場合は複勝のみ、２頭以上の場合はワイドで購入するようにしてみます。

In [61]:
# 3頭以内に入る確率が0.7と0.8以上のデータをそれぞれ抽出
data_pred07 = simulation_data[simulation_data['pred'] > 0.7]
data_pred08 = simulation_data[simulation_data['pred'] > 0.8]
print("データ数_0.7以上", len(data_pred07))
print("データ数_0.8以上", len(data_pred08))

データ数_0.7以上 135
データ数_0.8以上 46


In [62]:
# それぞれのレースごとに予測対象の馬が何頭いるか確認
data_pred07["race_id"].value_counts()

race_id
202407030706    2
202404030809    2
202407030301    2
202406040109    2
202406040103    2
               ..
202407030206    1
202407030205    1
202407030204    1
202407030203    1
202407030908    1
Name: count, Length: 122, dtype: int64

In [63]:
data_pred08["race_id"].value_counts()

race_id
202401020801    1
202407030606    1
202407030402    1
202407030405    1
202407030409    1
202406040501    1
202406040504    1
202406040506    1
202406040601    1
202406040602    1
202406040609    1
202407030608    1
202404030803    1
202406040702    1
202406040711    1
202406040712    1
202407030704    1
202407030709    1
202406040804    1
202406040809    1
202407030812    1
202406040902    1
202407030310    1
202407030301    1
202407030209    1
202407030207    1
202404030805    1
202404030806    1
202404030807    1
202404030808    1
202404030809    1
202406040103    1
202407030102    1
202407030103    1
202407030104    1
202407030106    1
202407030111    1
202406040201    1
202406040202    1
202406040204    1
202406040212    1
202407030201    1
202407030203    1
202407030204    1
202407030206    1
202406040909    1
Name: count, dtype: int64

#### predが0.7以上の場合

In [64]:
# race_idごとにグループ化し、そのサイズを計算
grouped = data_pred07.groupby('race_id').size()

# 行数が1のrace_idを抽出
one_row_race_ids = grouped[grouped == 1].index

# 行数が2のrace_idを抽出
two_row_race_ids = grouped[grouped == 2].index

# 該当する行を抽出
one_row_race_df = data_pred07[data_pred07['race_id'].isin(one_row_race_ids)]
two_row_race_df = data_pred07[data_pred07['race_id'].isin(two_row_race_ids)]

In [65]:
# 予測が1になっているデータのみを取得する
data = two_row_race_df[two_row_race_df["pred_class"] == 1]

# 結果データの race_id のリストを取得
result_race_ids = data['race_id'].unique()

# 払戻金データを結果データの race_id のみにフィルタリング
filtered_return_tables = pay_result[pay_result['race_id'].isin(result_race_ids)]

# ワイドの回収率
recovery_rate(data, filtered_return_tables)

的中したレース
レースID: 202404030809, 的中馬番: [1, 7], 回収金: 120
レースID: 202406040101, 的中馬番: [5, 8], 回収金: 160
レースID: 202406040109, 的中馬番: [1, 2], 回収金: 110
レースID: 202406040201, 的中馬番: [4, 8], 回収金: 110
レースID: 202407030103, 的中馬番: [2, 3], 回収金: 100
レースID: 202407030301, 的中馬番: [3, 8], 回収金: 200
レースID: 202407030706, 的中馬番: [3, 7], 回収金: 140
----------------------------------------------------------------------------------------------------
予測対象レース数 13
----------------------------------------------------------------------------------------------------
予測対象馬券数 13
----------------------------------------------------------------------------------------------------
的中数 7
----------------------------------------------------------------------------------------------------
的中率 53.84615384615385%
----------------------------------------------------------------------------------------------------
回収金額 940
----------------------------------------------------------------------------------------------------
回収率 72.3076923076

In [66]:
# 予測が1になっているデータのみを取得する
data = one_row_race_df[one_row_race_df["pred_class"] == 1]

# 結果データの race_id のリストを取得
result_race_ids = data['race_id'].unique()

# 払戻金データを結果データの race_id のみにフィルタリング
filtered_return_tables = pay_result[pay_result['race_id'].isin(result_race_ids)]

# 複勝の回収率
recovery_rate(data, filtered_return_tables, bet="fukusho")

的中したレース
レースID: 202401020801, 的中馬番: [8], 回収金: 110
レースID: 202401020807, 的中馬番: [11], 回収金: 140
レースID: 202401020809, 的中馬番: [4], 回収金: 120
レースID: 202404030801, 的中馬番: [4], 回収金: 110
レースID: 202404030802, 的中馬番: [8], 回収金: 130
レースID: 202404030803, 的中馬番: [14], 回収金: 110
レースID: 202404030805, 的中馬番: [3], 回収金: 110
レースID: 202404030806, 的中馬番: [8], 回収金: 110
レースID: 202404030807, 的中馬番: [4], 回収金: 110
レースID: 202404030808, 的中馬番: [8], 回収金: 110
レースID: 202406040204, 的中馬番: [15], 回収金: 110
レースID: 202406040205, 的中馬番: [3], 回収金: 120
レースID: 202406040211, 的中馬番: [10], 回収金: 110
レースID: 202406040212, 的中馬番: [12], 回収金: 110
レースID: 202406040301, 的中馬番: [7], 回収金: 110
レースID: 202406040302, 的中馬番: [3], 回収金: 110
レースID: 202406040304, 的中馬番: [15], 回収金: 120
レースID: 202406040305, 的中馬番: [5], 回収金: 110
レースID: 202406040501, 的中馬番: [6], 回収金: 110
レースID: 202406040503, 的中馬番: [7], 回収金: 110
レースID: 202406040504, 的中馬番: [2], 回収金: 110
レースID: 202406040505, 的中馬番: [8], 回収金: 120
レースID: 202406040506, 的中馬番: [8], 回収金: 100
レースID: 202406040509, 的中馬番: [14], 回収金: 110
レ

#### predが0.8以上の場合

In [67]:
# race_idごとにグループ化し、そのサイズを計算
grouped = data_pred08.groupby('race_id').size()

# 行数が1のrace_idを抽出
one_row_race_ids = grouped[grouped == 1].index

# 行数が2のrace_idを抽出
two_row_race_ids = grouped[grouped == 2].index

# 該当する行を抽出
one_row_race_df = data_pred08[data_pred08['race_id'].isin(one_row_race_ids)]
two_row_race_df = data_pred08[data_pred08['race_id'].isin(two_row_race_ids)]

In [69]:
# 予測が1になっているデータのみを取得する
data = one_row_race_df[one_row_race_df["pred_class"] == 1]

# 結果データの race_id のリストを取得
result_race_ids = data['race_id'].unique()

# 払戻金データを結果データの race_id のみにフィルタリング
filtered_return_tables = pay_result[pay_result['race_id'].isin(result_race_ids)]

# 複勝の回収率
recovery_rate(data, filtered_return_tables, bet="fukusho")

的中したレース
レースID: 202401020801, 的中馬番: [8], 回収金: 110
レースID: 202404030803, 的中馬番: [14], 回収金: 110
レースID: 202404030805, 的中馬番: [3], 回収金: 110
レースID: 202404030806, 的中馬番: [8], 回収金: 110
レースID: 202404030807, 的中馬番: [4], 回収金: 110
レースID: 202404030808, 的中馬番: [8], 回収金: 110
レースID: 202404030809, 的中馬番: [1], 回収金: 110
レースID: 202406040103, 的中馬番: [6], 回収金: 110
レースID: 202406040201, 的中馬番: [4], 回収金: 110
レースID: 202406040202, 的中馬番: [6], 回収金: 110
レースID: 202406040204, 的中馬番: [15], 回収金: 110
レースID: 202406040212, 的中馬番: [12], 回収金: 110
レースID: 202406040501, 的中馬番: [6], 回収金: 110
レースID: 202406040504, 的中馬番: [2], 回収金: 110
レースID: 202406040506, 的中馬番: [8], 回収金: 100
レースID: 202406040601, 的中馬番: [9], 回収金: 100
レースID: 202406040602, 的中馬番: [10], 回収金: 110
レースID: 202406040609, 的中馬番: [1], 回収金: 110
レースID: 202406040702, 的中馬番: [2], 回収金: 110
レースID: 202406040711, 的中馬番: [4], 回収金: 110
レースID: 202406040712, 的中馬番: [6], 回収金: 110
レースID: 202406040804, 的中馬番: [8], 回収金: 110
レースID: 202406040809, 的中馬番: [3], 回収金: 110
レースID: 202406040902, 的中馬番: [4], 回収金: 100
レースI

### まとめ
- モデルの予測に基づき全通りで購入した場合、複勝とワイドで控除率に近い回収率を達成できることが確認されました。

- predの確率を0.7以上、0.8以上に絞ることで的中率は向上しましたが、配当金がとても安い馬券のみとなったため、回収率はそれほど高くなりませんでした。

- データとしてはrace_resultテーブルしか利用していないため、過去のレース結果やスピード指数などのデータを組み合わせることで、より精度の高いモデルを作成し、これらの数値を改善する余地があると思われます。